In [1]:
import datetime
from xml.etree import ElementTree as ET
import requests
from requests.auth import HTTPDigestAuth
import numpy as np
import time
import json

In [2]:
# Most likely will need to add more key variables. Need to move two CT's from the 480 circuit to the 240V and then we can work only with one e-gauge

# If power is negative it means it is consuming power from grid
class EgaugeInterface():
    
    def __init__(self, url = None, username = None, password = None, t_sample = 5):
        # Initializing credentials
        self.url = url
        self.username = username
        self.password = password
        
        # Initializing parameters
        self.t_sample = t_sample
        self.keys = ['L1 Voltage', 'L2 Voltage', 'Power Circuit 1', 'Power Circuit 1*', 'Power Circuit 2',
                'Power Circuit 2*', 'Power Circuit 1 neutral', 'Shed Power', 'Control Fan Power', 'Control Fan Power*', 'ts']
        
        
    # Function to get and format e-gauge data
    def get_egauge_data(self, request):
        power_values = dict.fromkeys(self.keys, None)
        root = ET.fromstring(request.text)
        timestamp  = root.findtext("ts")
        
        if timestamp != None:
            for r in root.findall('r'):
                for child in r:
                    if r.get('n') == 'L1 Voltage':
                        power_values['L1 Voltage'] = (int(child.text))
                    elif r.get('n') == 'L2 Voltage':
                        power_values['L2 Voltage'] = (int(child.text))
                    elif r.get('n') == 'Power Circuit 1':
                        power_values['Power Circuit 1'] = (int(child.text))
                    elif r.get('n') == 'Power Circuit 1*':
                        power_values['Power Circuit 1*'] = (int(child.text))
                    elif r.get('n') == 'Power Circuit 2':
                        power_values['Power Circuit 2'] = (int(child.text))
                    elif r.get('n') == 'Power Circuit 2*':
                        power_values['Power Circuit 2*'] = (int(child.text))
                    elif r.get('n') == 'Power Circuit 1 neutral':
                        power_values['Power Circuit 1 neutral'] = (int(child.text))
                    elif r.get('n') == 'Shed Power':
                        power_values['Shed Power'] = (int(child.text))
                    elif r.get('n') == 'Control Fan Power':
                        power_values['Control Fan Power'] = (int(child.text))
                    elif r.get('n') == 'Control Fan Power*':
                        power_values['Control Fan Power*'] = (int(child.text))

            power_values['ts'] = int(timestamp)
        
        else:
            print('No values from request')
            
        return power_values

    
    # Function to process data from e-gauge and convert to useful power values
    def processing_egauge_data(self):
        power_values = dict.fromkeys(self.keys, None)
        try:
            resp = requests.get(self.url, auth=HTTPDigestAuth(self.username, self.password))
            resp.raise_for_status()
            data_ini = self.get_egauge_data(resp)
      
        except requests.exceptions.HTTPError as err:
            print(err)
            return json.dumps(power_values)

        time.sleep(self.t_sample)

        try:
            resp = requests.get(self.url, auth=HTTPDigestAuth(self.username, self.password))
            resp.raise_for_status()
            data_end = self.get_egauge_data(resp)

        except requests.exceptions.HTTPError as err:
            print(err)
            return json.dumps(power_values)

        
        ts_delta = data_end['ts'] - data_ini['ts']
        try:
            power_values['ts'] = datetime.datetime.fromtimestamp(int(data_end['ts'])).strftime('%Y-%m-%d %H:%M:%S')
            power_values['L1 Voltage'] = ((data_end['L1 Voltage'] - data_ini['L1 Voltage'])/ts_delta)/1000
            power_values['L2 Voltage'] = ((data_end['L2 Voltage'] - data_ini['L2 Voltage'])/ts_delta)/1000
            power_values['Power Circuit 1'] = (data_end['Power Circuit 1'] - data_ini['Power Circuit 1'])/ts_delta
            power_values['Power Circuit 1*'] = (data_end['Power Circuit 1*'] - data_ini['Power Circuit 1*'])/ts_delta
            power_values['Power Circuit 2'] = (data_end['Power Circuit 2'] - data_ini['Power Circuit 2'])/ts_delta
            power_values['Power Circuit 2*'] = (data_end['Power Circuit 2*'] - data_ini['Power Circuit 2*'])/ts_delta
            power_values['Power Circuit 1 neutral'] = (data_end['Power Circuit 1 neutral'] - data_ini['Power Circuit 1 neutral'])/ts_delta
            power_values['Shed Power'] = (data_end['Shed Power'] - data_ini['Shed Power'])/ts_delta
            power_values['Control Fan Power'] = (data_end['Control Fan Power'] - data_ini['Control Fan Power'])/ts_delta
            power_values['Control Fan Power*'] = (data_end['Control Fan Power*'] - data_ini['Control Fan Power*'])/ts_delta

            json_dict = json.dumps(power_values)
            print('Data Dict: ', json_dict)
            return json.dumps(power_values)
            
        except Exception as e:
            print('Error retrieving data from E-Gauge API: ', e)
            return None


In [3]:
# url = 'http://stanford002.egaug.es/cgi-bin/egauge?param'
# user = 'test'
# password = 'testhh'

# user480 = 'owner'
# password480 = 'szk87P57GW'
# url480 = 'https://egauge48709.egaug.es/cgi-bin/egauge?param'

user240 = 'owner'
password240 = 'SuXbc58vi9'
url240 = 'https://egauge46613.egaug.es/cgi-bin/egauge'

power = EgaugeInterface(url=url240, username=user240, password=password240).processing_egauge_data()
# power = EgaugeInterface(url=url480, username=user480, password=password480).processing_egauge_data()

Error retrieving data from E-Gauge API:  unsupported operand type(s) for -: 'NoneType' and 'NoneType'


In [11]:
power

'{"L1 Voltage": 236.823, "L2 Voltage": 237.981, "Power Circuit 1": 2738.0, "Power Circuit 1*": 6208.0, "Power Circuit 2": 2385.0, "Power Circuit 2*": 4746.0, "Power Circuit 1 neutral": 2463.0, "Shed Power": -58.0, "Control Fan Power": 9216.0, "Control Fan Power*": 19753.0, "ts": "2019-06-24 17:46:20"}'

In [13]:
int(2.3)

2

In [28]:
a = json.dumps(None)

TypeError: Object of type set is not JSON serializable

In [25]:
a

'null'

In [ ]:
(167538791)

In [1]:
b=[0,1]
for i in b:
    print(i)

0
1
